# Lab 6

You are tasked with evaluating card counting strategies for black jack. In order to do so, you will use object oriented programming to create a playable casino style black jack game where a computer dealer plays against $n$ computer players and possibily one human player. If you don't know the rules of blackjack or card counting, please google it. 

A few requirements:
* The game should utilize multiple 52-card decks. Typically the game is played with 6 decks.
* Players should have chips.
* Dealer's actions are predefined by rules of the game (typically hit on 16). 
* The players should be aware of all shown cards so that they can count cards.
* Each player could have a different strategy.
* The system should allow you to play large numbers of games, study the outcomes, and compare average winnings per hand rate for different strategies.

1. Begin by creating a classes to represent cards and decks. The deck should support more than one 52-card set. The deck should allow you to shuffle and draw cards. Include a "plastic" card, placed randomly in the deck. Later, when the plastic card is dealt, shuffle the cards before the next deal.

In [31]:
import random

class Card:
    def __init__(self, value, suit):
        self.value = value
        self.suit = suit

    def get_value(self):
        if self.value in ['Jack', 'Queen', 'King']:
            return 10
        if self.value == 'Ace':
            return 11
        if self.value == 'Plastic':
            return 0 
        return int(self.value)

    def __str__(self):
        return f"{self.value} of {self.suit}"
        
class Deck:
    def __init__(self, num_decks=1):
        self.num_decks = num_decks
        self.cards = self.create_deck()
        self.add_plastic_card()
        self.shuffle()

    def create_deck(self):
        values = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'Jack', 'Queen', 'King', 'Ace']
        suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
        return [Card(value, suit) for _ in range(self.num_decks) for value in values for suit in suits]

    def add_plastic_card(self):
        self.plastic_card = Card('Plastic', 'Special')
        self.cards.append(self.plastic_card)

    def shuffle(self):
        random.shuffle(self.cards)

    def draw_card(self):
        if not self.cards:
            print("No more cards in the deck, reshuffling...")
            self.reset_deck()
        drawn_card = self.cards.pop()
        if drawn_card.value == 'Plastic':
            print("Plastic card drawn! Shuffling the deck.")
            self.shuffle()
        return drawn_card

    def reset_deck(self):
        self.cards = self.create_deck()
        self.add_plastic_card()
        self.shuffle()

card1=Card('Jack', 'Spades')
print(card1.get_value())

deck=Deck(num_decks=3)
deck.shuffle()
drawn = deck.draw_card()
print(drawn)


10
2 of Spades


2. Now design your game on a UML diagram. You may want to create classes to represent, players, a hand, and/or the game. As you work through the lab, update your UML diagram. At the end of the lab, submit your diagram (as pdf file) along with your notebook. 

3. Begin with implementing the skeleton (ie define data members and methods/functions, but do not code the logic) of the classes in your UML diagram.

4. Complete the implementation by coding the logic of all functions. For now, just implement the dealer player and human player.

In [28]:
class Hand:
    def __init__(self):
        self.cards = []

    def add_card(self, card):
        self.cards.append(card)

    def hand_total(self):
        total_value = sum(card.get_value() for card in self.cards)
        num_aces = sum(1 for card in self.cards if card.value == 'Ace')
        while total_value > 21 and num_aces:
            total_value -= 10
            num_aces -= 1
        return total_value

    def __str__(self):
        return ', '.join(str(card) for card in self.cards)
        
class Player:
    def __init__(self, name, chips):
        self.name = name
        self.chips = chips
        self.hand = Hand()

    def hit(self, deck):
        card = deck.draw_card()
        self.hand.add_card(card)
        print(f"{self.name} hits and gets {card}")

    def stand(self):
        print(f"{self.name} stands with hand: {self.hand} (Value: {self.calculate_hand_total()})")

    def calculate_hand_total(self):
        return self.hand.hand_total()

class BJPlayer(Player):
    def __init__(self, name, chips):
        super().__init__(name, chips)

    def make_move(self, deck):
        while self.calculate_hand_total() <= 16:
            self.hit(deck)
            if self.calculate_hand_total() > 21:
                print(f"{self.name} busts with hand: {self.hand} (Value: {self.calculate_hand_total()})")
                break
        if self.calculate_hand_total() <= 21:
            self.stand()

    def win_bet(self):
        self.chips += self.current_bet * 2
        self.current_bet = 0 

    def lose_bet(self):
        self.current_bet = 0 

    def tie_bet(self):
        self.chips += self.current_bet
        self.current_bet = 0  

    def make_move(self, deck):
        while self.calculate_hand_total() <= 16:  
            self.hit(deck)
            if self.calculate_hand_total() > 21: 
                print(f"{self.name} busts with hand: {self.hand} (Value: {self.calculate_hand_total()})")
                break
        if self.calculate_hand_total() <= 21:
            self.stand()


class Dealer(Player):
    def __init__(self):
        super().__init__("Dealer", chips=0)

    def play(self, deck):
        print("Dealer's turn:")
        while self.calculate_hand_total() < 17:
            self.hit(deck)
        print(f"Dealer stands with hand: {self.hand} (Value: {self.calculate_hand_total()})")



5.  Test. Demonstrate game play. For example, create a game of several dealer players and show that the game is functional through several rounds.

In [32]:
class Game:
    def __init__(self):
        self.deck = Deck(num_decks=3)
        self.players = []
        self.dealer = Dealer()

    def add_player(self, name, chips):
        self.players.append(BJPlayer(name, chips))

    def start_game(self):
        for player in self.players:
            player.hand = Hand()  
            player.hit(self.deck)
            player.hit(self.deck)
        self.dealer.hand = Hand()  
        self.dealer.hit(self.deck)
        self.dealer.hit(self.deck)

    def play_round(self):
        for player in self.players:
            player.make_move(self.deck)
        self.dealer.play(self.deck)
        self.determine_winner()

    def determine_winner(self):
        dealer_value = self.dealer.calculate_hand_total()
        print(f"Dealer's hand value: {dealer_value}")
        for player in self.players:
            player_value = player.calculate_hand_total()
            if player_value > 21:
                print(f"{player.name} busts! Dealer wins.")
            elif dealer_value > 21 or player_value > dealer_value:
                print(f"{player.name} wins!")
                player.chips += 10  # Win chips
            elif player_value < dealer_value:
                print(f"Dealer wins against {player.name}.")
                player.chips -= 10  # Lose chips
            else:
                print(f"{player.name} ties with the dealer.")

def start_game():
    game = Game()
    
    # Adding players
    game.add_player("Derrick", 10000)
    game.add_player("Sally", 10000)
    game.add_player("Alice", 10000)

    rounds = 5 
    for round_number in range(1, rounds + 1):
        print(f"\n--- Round {round_number} ---")
        game.start_game()
        game.play_round()
        for player in game.players:
            print(f"{player.name} has {player.chips} chips.")

if __name__ == "__main__":
    start_game()


--- Round 1 ---
Derrick hits and gets 2 of Hearts
Derrick hits and gets 6 of Spades
Sally hits and gets Queen of Spades
Sally hits and gets Ace of Spades
Alice hits and gets 9 of Hearts
Alice hits and gets Jack of Hearts
Dealer hits and gets 3 of Spades
Dealer hits and gets 5 of Diamonds
Derrick hits and gets 10 of Diamonds
Derrick stands with hand: 2 of Hearts, 6 of Spades, 10 of Diamonds (Value: 18)
Sally stands with hand: Queen of Spades, Ace of Spades (Value: 21)
Alice stands with hand: 9 of Hearts, Jack of Hearts (Value: 19)
Dealer's turn:
Dealer hits and gets 5 of Clubs
Dealer hits and gets 5 of Clubs
Dealer stands with hand: 3 of Spades, 5 of Diamonds, 5 of Clubs, 5 of Clubs (Value: 18)
Dealer's hand value: 18
Derrick ties with the dealer.
Sally wins!
Alice wins!
Derrick has 10000 chips.
Sally has 10010 chips.
Alice has 10010 chips.

--- Round 2 ---
Derrick hits and gets 2 of Hearts
Derrick hits and gets Ace of Diamonds
Sally hits and gets Queen of Clubs
Sally hits and gets 7 o

6. Implement a new player with the following strategy:

    * Assign each card a value: 
        * Cards 2 to 6 are +1 
        * Cards 7 to 9 are 0 
        * Cards 10 through Ace are -1
    * Compute the sum of the values for all cards seen so far.
    * Hit if sum is very negative, stay if sum is very positive. Select a threshold for hit/stay, e.g. 0 or -2.  

In [27]:
class StrategyPlayer(Player):
    def __init__(self, name, chips):
        super().__init__(name, chips)
        self.count = 0

    def update_count(self, card):
        if card.value in ['2', '3', '4', '5', '6']:
            self.count += 1
        elif card.value in ['10', 'Jack', 'Queen', 'King', 'Ace']:
            self.count -= 1

    def make_move(self, deck, threshold=-2):
        print(f"{self.name}'s current count: {self.count}")
        if self.count < threshold:
            self.hit(deck)
        else:
            self.stand()

7. Create a test scenario where one player, using the above strategy, is playing with a dealer and 3 other players that follow the dealer's strategy. Each player starts with same number of chips. Play 50 rounds (or until the strategy player is out of money). Compute the strategy player's winnings. You may remove unnecessary printouts from your code (perhaps implement a verbose/quiet mode) to reduce the output.

In [30]:
import random

class Card:
    def __init__(self, value, suit):
        self.value = value
        self.suit = suit

    def get_value(self):
        if self.value in ['Jack', 'Queen', 'King']:
            return 10
        if self.value == 'Ace':
            return 11
        if self.value == 'Plastic':
            return 0  
        return int(self.value)

    def __str__(self):
        return f"{self.value} of {self.suit}"

class Deck:
    def __init__(self, num_decks=1):
        self.num_decks = num_decks
        self.cards = self.create_deck()
        self.add_plastic_card()
        self.shuffle()

    def create_deck(self):
        values = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'Jack', 'Queen', 'King', 'Ace']
        suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
        return [Card(value, suit) for _ in range(self.num_decks) for value in values for suit in suits]

    def add_plastic_card(self):
        self.plastic_card = Card('Plastic', 'Special')
        self.cards.append(self.plastic_card)

    def shuffle(self):
        random.shuffle(self.cards)

    def draw_card(self):
        if not self.cards:
            print("No more cards in the deck, reshuffling...")
            self.reset_deck()
        drawn_card = self.cards.pop()
        if drawn_card.value == 'Plastic':
            print("Plastic card drawn! Shuffling the deck.")
            self.shuffle()
        return drawn_card

    def reset_deck(self):
        self.cards = self.create_deck()
        self.add_plastic_card()
        self.shuffle()

class Hand:
    def __init__(self):
        self.cards = []

    def add_card(self, card):
        self.cards.append(card)

    def hand_total(self):
        total_value = sum(card.get_value() for card in self.cards)
        num_aces = sum(1 for card in self.cards if card.value == 'Ace')
        while total_value > 21 and num_aces:
            total_value -= 10
            num_aces -= 1
        return total_value

    def __str__(self):
        return ', '.join(str(card) for card in self.cards)

class Player:
    def __init__(self, name, chips):
        self.name = name
        self.chips = chips
        self.hand = Hand()

    def hit(self, deck):
        card = deck.draw_card()
        self.hand.add_card(card)
        print(f"{self.name} hits and gets {card}")

    def stand(self):
        print(f"{self.name} stands with hand: {self.hand} (Value: {self.calculate_hand_total()})")

    def calculate_hand_total(self):
        return self.hand.hand_total()

class BJPlayer(Player):
    def __init__(self, name, chips):
        super().__init__(name, chips)

    def make_move(self, deck):
        while self.calculate_hand_total() <= 16:
            self.hit(deck)
            if self.calculate_hand_total() > 21:
                print(f"{self.name} busts with hand: {self.hand} (Value: {self.calculate_hand_total()})")
                break
        if self.calculate_hand_total() <= 21:
            self.stand()

class Dealer(Player):
    def __init__(self):
        super().__init__("Dealer", chips=0)

    def play(self, deck):
        print("Dealer's turn:")
        while self.calculate_hand_total() < 17:
            self.hit(deck)
        print(f"Dealer stands with hand: {self.hand} (Value: {self.calculate_hand_total()})")

class Game:
    def __init__(self):
        self.deck = Deck(num_decks=3)
        self.players = []
        self.dealer = Dealer()

    def add_player(self, name, chips):
        self.players.append(BJPlayer(name, chips))

    def start_game(self):
        for player in self.players:
            player.hand = Hand()
            player.hit(self.deck)
            player.hit(self.deck)
        self.dealer.hand = Hand()  
        self.dealer.hit(self.deck)
        self.dealer.hit(self.deck)

    def play_round(self):
        for player in self.players:
            player.make_move(self.deck)
        self.dealer.play(self.deck)
        self.determine_winner()

    def determine_winner(self):
        dealer_value = self.dealer.calculate_hand_total()
        print(f"Dealer's hand value: {dealer_value}")
        for player in self.players:
            player_value = player.calculate_hand_total()
            if player_value > 21:
                print(f"{player.name} busts! Dealer wins.")
            elif dealer_value > 21 or player_value > dealer_value:
                print(f"{player.name} wins!")
                player.chips += 10  # Win chips
            elif player_value < dealer_value:
                print(f"Dealer wins against {player.name}.")
                player.chips -= 10  # Lose chips
            else:
                print(f"{player.name} ties with the dealer.")

def start_game():
    game = Game()
    
    # Adding players
    game.add_player("Derrick", 10000)
    game.add_player("Sally", 10000)
    game.add_player("Alice", 10000)

    rounds = 50 
    for round_number in range(1, rounds + 1):
        print(f"\n--- Round {round_number} ---")
        game.start_game()
        game.play_round()
        for player in game.players:
            print(f"{player.name} has {player.chips} chips.")

if __name__ == "__main__":
    start_game()



--- Round 1 ---
Derrick hits and gets Jack of Diamonds
Derrick hits and gets King of Diamonds
Sally hits and gets King of Clubs
Sally hits and gets 6 of Clubs
Alice hits and gets 8 of Spades
Alice hits and gets 3 of Hearts
Dealer hits and gets 7 of Clubs
Dealer hits and gets 3 of Spades
Derrick stands with hand: Jack of Diamonds, King of Diamonds (Value: 20)
Sally hits and gets 8 of Spades
Sally busts with hand: King of Clubs, 6 of Clubs, 8 of Spades (Value: 24)
Alice hits and gets 4 of Spades
Alice hits and gets 5 of Spades
Alice stands with hand: 8 of Spades, 3 of Hearts, 4 of Spades, 5 of Spades (Value: 20)
Dealer's turn:
Dealer hits and gets 2 of Diamonds
Dealer hits and gets 7 of Hearts
Dealer stands with hand: 7 of Clubs, 3 of Spades, 2 of Diamonds, 7 of Hearts (Value: 19)
Dealer's hand value: 19
Derrick wins!
Sally busts! Dealer wins.
Alice wins!
Derrick has 10010 chips.
Sally has 10000 chips.
Alice has 10010 chips.

--- Round 2 ---
Derrick hits and gets 5 of Diamonds
Derrick h

8. Create a loop that runs 100 games of 50 rounds, as setup in previous question, and store the strategy player's chips at the end of the game (aka "winnings") in a list. Histogram the winnings. What is the average winnings per round? What is the standard deviation. What is the probabilty of net winning or lossing after 50 rounds?


In [45]:
import random
import numpy as np

class Card:
    def __init__(self, value, suit):
        self.value = value
        self.suit = suit

    def get_value(self):
        if self.value in ['Jack', 'Queen', 'King']:
            return 10
        elif self.value == 'Ace':
            return 11
        else:
            return int(self.value)

    def __str__(self):
        return f"{self.value} of {self.suit}"

class Deck:
    def __init__(self, num_decks=6):
        self.num_decks = num_decks
        self.cards = self.create_deck()
        self.add_plastic_card()
        self.shuffle()

    def create_deck(self):
        values = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'Jack', 'Queen', 'King', 'Ace']
        suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
        return [Card(value, suit) for i in range(self.num_decks) for value in values for suit in suits]

    def add_plastic_card(self):
        self.plastic_card = Card('Plastic', 'Special')
        self.cards.append(self.plastic_card)

    def shuffle(self):
        random.shuffle(self.cards)

    def draw_card(self):
        if len(self.cards) == 0:
            self.reset_deck()

        drawn_card = self.cards.pop()
        
        if drawn_card.value == 'Plastic':
            print("Plastic card drawn! Shuffling the deck.")
            self.shuffle()

        return drawn_card

    def reset_deck(self):
        self.cards = self.create_deck()
        self.add_plastic_card()
        self.shuffle()

class Hand:
    def __init__(self):
        self.cards = []

    def add_card(self, card):
        self.cards.append(card)

    def hand_total(self):
        total_value = sum(card.get_value() for card in self.cards)
        num_aces = sum(1 for card in self.cards if card.value == 'Ace')
        while total_value > 21 and num_aces:
            total_value -= 10
            num_aces -= 1
        return total_value

    def __str__(self):
        return ', '.join(str(card) for card in self.cards)

class Player:
    def __init__(self, name, chips):
        self.name = name
        self.chips = chips
        self.hand = Hand()

    def hit(self, deck):
        card = deck.draw_card()
        self.hand.add_card(card)
        print(f"{self.name} hits and gets {card}")

    def stand(self):
        print(f"{self.name} stands with {self.hand}")

    def calculate_hand_total(self):
        return self.hand.hand_total()

    def reset_hand(self):
        self.hand = Hand()

class BJPlayer(Player):
    def __init__(self, name, chips):
        super().__init__(name, chips)
        self.current_bet = 10 

    def place_bet(self):
        if self.current_bet > self.chips:
            print("Not enough chips to place bet.")
            return False
        self.chips -= self.current_bet
        return True

    def win_bet(self):
        self.chips += self.current_bet * 2
        self.current_bet = 10

    def lose_bet(self):
        self.current_bet = 10

    def tie_bet(self):
        self.chips += self.current_bet
        self.current_bet = 10

    def make_move(self):
        if self.calculate_hand_total() <= 16:
            return 'hit'
        return 'stand'

class Dealer(Player):
    def __init__(self):
        super().__init__("Dealer", 0)

    def play(self, deck):
        while self.calculate_hand_total() < 17:
            self.hit(deck)

class Game:
    def __init__(self, verbose=True):
        self.deck = Deck(num_decks=6)
        self.players = []
        self.dealer = Dealer()
        self.verbose = verbose

    def add_player(self, name, chips):
        self.players.append(BJPlayer(name, chips))

    def start_game(self):
        for player in self.players:
            player.reset_hand()
            player.hit(self.deck)
            player.hit(self.deck)
        self.dealer.reset_hand()
        self.dealer.hit(self.deck)
        self.dealer.hit(self.deck)

    def play_round(self):
        for player in self.players:
            while True:
                if self.verbose:
                    print(f"{player.name}'s hand: {player.hand} (Value: {player.calculate_hand_total()})")
                if player.calculate_hand_total() >= 21:
                    if self.verbose:
                        print(f"{player.name} busts!")
                    break
                action = player.make_move()
                if action == 'hit':
                    player.hit(self.deck)
                elif action == 'stand':
                    if self.verbose:
                        print(f"{player.name} stands.")
                    break
        
        self.dealer.play(self.deck)
        self.determine_winner()

    def determine_winner(self):
        dealer_value = self.dealer.calculate_hand_total()
        if self.verbose:
            print(f"Dealer's hand value: {dealer_value}")

        for player in self.players:
            player_value = player.calculate_hand_total()
            if player_value > 21:
                if self.verbose:
                    print(f"{player.name} busts! Dealer wins.")
            elif dealer_value > 21 or player_value > dealer_value:
                player.win_bet()
                if self.verbose:
                    print(f"{player.name} wins!")
            elif player_value < dealer_value:
                player.lose_bet()
                if self.verbose:
                    print(f"Dealer wins against {player.name}.")
            else:
                player.tie_bet()
                if self.verbose:
                    print(f"{player.name} ties with the dealer.")

def run_simulation(num_games=100, rounds=50, initial_chips=10000):
    strategy_player_winnings = []

    for game_number in range(num_games):
        game = Game()
        game.add_player("Strategy Player", initial_chips)
        game.add_player("Derrick", initial_chips)
        game.add_player("Sally", initial_chips)
        game.add_player("Alice", initial_chips)

        for round_number in range(rounds):
            game.start_game()
            game.play_round()

        strategy_player = game.players[0]
        strategy_player_winnings.append(strategy_player.chips - initial_chips)

    return strategy_player_winnings

def analyze_results(winnings):
    average_winnings = np.mean(winnings)
    std_deviation = np.std(winnings)
    net_wins = sum(1 for win in winnings if win > 0)
    net_losses = sum(1 for win in winnings if win < 0)
    total_games = len(winnings)

    win_probability = net_wins / total_games
    loss_probability = net_losses / total_games

    print(f"Average winnings per game: {average_winnings:.2f}")
    print(f"Standard deviation of winnings: {std_deviation:.2f}")
    print(f"Probability of net winning: {win_probability:.2f}")
    print(f"Probability of net losing: {loss_probability:.2f}")

if __name__ == "__main__":
    analyze_results(winnings)

Average winnings per game: 101.00
Standard deviation of winnings: 57.44
Probability of net winning: 0.95
Probability of net losing: 0.03


9. Repeat previous questions scanning the value of the threshold. Try at least 5 different threshold values. Can you find an optimal value?

In [55]:
def simulate_games(num_games=100, rounds=50, threshold=0, verbose=False):
    winnings = []
    for _ in range(num_games):
        game = Game(verbose=verbose)
        strategy_player = BJPlayer("Strategy Player", 10000)
        game.players.append(strategy_player)
        for i in range(3):
            game.add_player(f"Player {i+1}", 10000)
        
        strategy_player_chips = 10000
        for _ in range(rounds):
            game.start_game()
            game.play_round()
            # Adjust chips based on results
            if strategy_player.chips > 10000: 
                strategy_player_chips += 10
            elif strategy_player.chips < 10000: 
                strategy_player_chips -= 10
        winnings.append(strategy_player_chips)

    return winnings

def analyze_results(winnings):
    avg_winnings = np.mean(winnings)
    std_dev = np.std(winnings)
    net_wins = sum(1 for win in winnings if win > 10000)
    net_losses = sum(1 for win in winnings if win < 10000)
    prob_win = net_wins / len(winnings)
    prob_loss = net_losses / len(winnings)

    return avg_winnings - 10000, std_dev, prob_win, prob_loss

if __name__ == "__main__":
    thresholds = [-3, -2, -1, 0, 1]
    results = {}

    for threshold in thresholds:
        print(f"Running simulations with threshold: {threshold}")
        winnings = simulate_games(num_games=100, rounds=50, threshold=threshold, verbose=False)
        results[threshold] = analyze_results(winnings)

    for threshold, (avg_win, std_dev, prob_win, prob_loss) in results.items():
        print(f"Threshold: {threshold}, Avg Winnings: {avg_win:.2f}, Std Dev: {std_dev:.2f}, "
              f"Probability of Winning: {prob_win:.2f}, Probability of Losing: {prob_loss:.2f}")


Running simulations with threshold: -3
Strategy Player hits and gets Queen of Hearts
Strategy Player hits and gets 6 of Clubs
Player 1 hits and gets 7 of Spades
Player 1 hits and gets Ace of Hearts
Player 2 hits and gets 9 of Spades
Player 2 hits and gets 6 of Hearts
Player 3 hits and gets 10 of Diamonds
Player 3 hits and gets 3 of Diamonds
Dealer hits and gets 8 of Hearts
Dealer hits and gets 8 of Hearts
Strategy Player hits and gets Jack of Clubs
Player 2 hits and gets 9 of Clubs
Player 3 hits and gets Ace of Spades
Player 3 hits and gets 4 of Spades
Dealer hits and gets Jack of Spades
Strategy Player hits and gets King of Clubs
Strategy Player hits and gets 2 of Diamonds
Player 1 hits and gets Queen of Hearts
Player 1 hits and gets 5 of Clubs
Player 2 hits and gets 5 of Diamonds
Player 2 hits and gets 2 of Spades
Player 3 hits and gets 5 of Hearts
Player 3 hits and gets 6 of Diamonds
Dealer hits and gets 9 of Hearts
Dealer hits and gets 3 of Hearts
Strategy Player hits and gets 3 of

ValueError: invalid literal for int() with base 10: 'Plastic'

10. Create a new strategy based on web searches or your own ideas. Demonstrate that the new strategy will result in increased or decreased winnings. 

In [50]:
class AggressivePlayer(BJPlayer):
    def __init__(self, name, chips):
        super().__init__(name, chips)
        self.count = 0 

    def update_count(self, card):
        if card.value in ['2', '3', '4', '5', '6']:
            self.count += 1
        elif card.value in ['10', 'Jack', 'Queen', 'King', 'Ace']:
            self.count -= 1

    def place_bet(self):
        if self.count > 0:
            self.current_bet = min(20, self.chips)
        else:
            self.current_bet = 10
        self.chips -= self.current_bet
        print(f"{self.name} bets {self.current_bet} chips.")

    def make_move(self, deck):
        while True:
            print(f"{self.name}'s hand: {self.hand} (Value: {self.calculate_hand_total()}, Count: {self.count})")
            if self.calculate_hand_total() >= 21:
                print(f"{self.name} busts!")
                break
            action = 'hit' if self.calculate_hand_total() <= 16 else 'stand'
            print(f"{self.name} chooses to {action}.")
            if action == 'hit':
                card = deck.draw_card()
                self.hit(deck)
                self.update_count(card)
            else:
                self.stand()
                break

def simulate_strategy(num_games, rounds_per_game):
    strategy_winnings = []

    for game_num in range(num_games):
        game = Game()
        strategy_player = AggressivePlayer("Strategy Player", 10000)
        game.add_player(strategy_player.name, strategy_player.chips)

        for _ in range(rounds_per_game):
            game.start_game()
            game.play_round()
            # Collect winnings after each round
            strategy_winnings.append(strategy_player.chips)

    return strategy_winnings

num_games = 5
rounds_per_game = 1
winnings = simulate_strategy(num_games, rounds_per_game)

average_winnings = sum(winnings) / len(winnings)
print(f"Average winnings after {num_games * rounds_per_game} rounds: {average_winnings}")


Strategy Player hits and gets 2 of Clubs
Strategy Player hits and gets King of Clubs
Dealer hits and gets 10 of Diamonds
Dealer hits and gets 10 of Clubs
Strategy Player's hand: 2 of Clubs, King of Clubs (Value: 12)
Strategy Player hits and gets 8 of Spades
Strategy Player's hand: 2 of Clubs, King of Clubs, 8 of Spades (Value: 20)
Strategy Player stands.
Dealer's hand value: 20
Strategy Player ties with the dealer.
Strategy Player hits and gets King of Hearts
Strategy Player hits and gets Jack of Hearts
Dealer hits and gets King of Diamonds
Dealer hits and gets 4 of Diamonds
Strategy Player's hand: King of Hearts, Jack of Hearts (Value: 20)
Strategy Player stands.
Dealer hits and gets 5 of Diamonds
Dealer's hand value: 19
Strategy Player wins!
Strategy Player hits and gets 7 of Hearts
Strategy Player hits and gets 2 of Clubs
Dealer hits and gets Ace of Clubs
Dealer hits and gets 4 of Diamonds
Strategy Player's hand: 7 of Hearts, 2 of Clubs (Value: 9)
Strategy Player hits and gets 5 of 